# Effective Dimension of Qiskit Neural Networks

In this tutorial, we will take advantage of the `EffectiveDimension` and `LocalEffectiveDimension` classes to evaluate the power of Quantum Neural Network models. These are metrics based on information geometry that connect to notions such as trainability, expressibility or ability to generalize.

Before diving into the code example, we will briefly explain what is the difference between these two metrics, and why are they relevant to the study of Quantum Neural Networks. More information about global effective dimension can be found in ['this paper'](https://arxiv.org/pdf/2011.00027.pdf), while the local effective dimension was introduced in a ['later work'](https://arxiv.org/abs/2112.04807), both by Abbas et al.

## 1. Global vs. Local Effective Dimension

Both classical and quantum machine learning models share a common goal: being good at **generalizing**, i.e. learning from data and applying these learnings on unseen data. Finding a good metric to assess this ability is a non-trivial matter. In [The Power of Quantum Neural Networks](https://arxiv.org/pdf/2011.00027.pdf), Abbas et al. introduce the **Global** Effective Dimension as a useful indicator of how well a particular model will be able to perform on new data.

Both the Global and Local Effective Dimension algorithms use the Fisher Information matrix to provide a measure of complexity. The details on how this matrix is calculated are provided in the [reference paper](https://arxiv.org/pdf/2011.00027.pdf), but in general terms, this matrix captures how sensitive a neural network's output is to changes in the network's parameter space.

The key difference between Global and Local Effective Dimension is actually not on the way they are computed, but in the nature of the parameter space that is analyzed. The global effective dimension incorporates the full parameter space of the model, and is calculated from a large number of parameters sets. However, the act of training restricts the number of parameters that the model has access to when applied to new data, and when looking into the ability to generalize of a model, it makes more sense to calculate a metric that is specific for a particular set of trained parameters. This is what is done with the Local Effective Dimension.


## 2. Basic Example (CircuitQNN)

This example shows how to set up a QNN model problem and run the effective dimension algorithm.



In [6]:
# Necessary imports
from qiskit.circuit.library import ZFeatureMap, RealAmplitudes
import matplotlib.pyplot as plt
import numpy as np

In [7]:
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit.utils import QuantumInstance
from qiskit import Aer, QuantumCircuit

In [8]:
from qiskit_machine_learning.algorithms.effective_dimension import EffectiveDimension, LocalEffectiveDimension

In [9]:
# declare quantum instance
qi_sv = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))

### 1. Create QNN

In [10]:
num_qubits = 3
# create a feature map
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)
# create a variational circuit
ansatz = RealAmplitudes(num_qubits, reps=1)

# create quantum circuit
qc = QuantumCircuit(num_qubits)
qc.append(feature_map, range(num_qubits))
qc.append(ansatz, range(num_qubits))
qc.decompose().draw( 'mpl')

In [21]:
# parity maps bitstrings to 0 or 1
def parity(x):
    return "{:b}".format(x).count("1") % 2
output_shape = 2  # corresponds to the number of classes, possible outcomes of the (parity) mapping.

In [ ]:
# construct QNN
qnn = CircuitQNN(
    qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=parity,
    output_shape=output_shape,
    sparse=False,
    quantum_instance=qi_sv
)

### 2. Define Problem

In order to define the problem, we need a series of sets of inputs and parameters, as well as the total number of data (n). The `inputs` and `params` are set in the class constructor, while the number of data is given during the call to the effective dimension computation to be able to test and compare how this measure changes with different dataset sizes.

In [24]:
# we can provide user-defined inputs and parameters
inputs = np.random.normal(0, 1, size=(10, qnn.num_inputs))
params = np.random.uniform(0, 1, size=(10, qnn.num_weights))

global_ed = EffectiveDimension(qnn=qnn,
                               params= params,
                               inputs = inputs)

In [25]:
# but we can also set the total number of input and parameter sets and these will be randomly selected for us:
num_inputs = 10
num_params = 10

global_ed = EffectiveDimension(qnn=qnn,
                               num_params=num_params,
                               num_inputs=num_inputs)

In [ ]:
# finally, we will define ranges to test different numbers of data, n
n = [5000, 8000, 10000, 40000, 60000, 100000, 150000, 200000, 500000, 1000000]

### 2. Create Global Effective Dimension Object

In [19]:
# if no inputs/parameters provided, they will be randomly sampled from a uniform/standard distribution
def callback(msg):
    print(msg)

global_ed = EffectiveDimension(qnn=qnn,
                               num_params=num_params,
                               num_inputs=num_inputs,
                               callback=callback)

### 3. Compute Effective Dimension

In [ ]:
global_eff_dim= global_ed.get_effective_dimension(n = n)
d = global_ed.num_weights()

iteration 0, time forward pass: 0.9238131046295166
iteration 0, time backward pass: 220.620502948761
iteration 1, time forward pass: 0.3635730743408203
iteration 1, time backward pass: 198.3666558265686
iteration 2, time forward pass: 0.2774331569671631
iteration 2, time backward pass: 217.56204390525818
iteration 3, time forward pass: 0.35566115379333496


In [ ]:
print("effdim: ", global_eff_dim)
# plot the normalised effective dimension for the model
plt.plot(n, np.array(global_eff_dim)/d)
plt.xlabel('number of data')
plt.ylabel('normalised effective dimension')
plt.show()

## 3. Other Examples

###  A) User-Defined Inputs and Params

In [ ]:
params = np.random.uniform(2, 2.5, size=(num_params, qnn.num_weights))
x = np.random.normal(0, 3, size=(num_inputs, qnn.num_inputs))

global_ed = EffectiveDimension(qnn=qnn,
                               params=params,
                               inputs=x)

In [ ]:
global_eff_dim, time = global_ed.eff_dim(n = n)
d = global_ed.d
print("effdim: ", global_eff_dim)
# plot the normalised effective dimension for the model
plt.plot(n, np.array(global_eff_dim)/d)
plt.xlabel('number of data')
plt.ylabel('normalised effective dimension')
plt.show()

###  B) Non-parity post-processing

In [ ]:
# construct QNN
qnn3 = CircuitQNN(
    qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    # interpret=parity,
    # output_shape=output_shape,
    sparse=False,
    quantum_instance=qi_sv
)

In [ ]:
global_ed_3 = EffectiveDimension(qnn=qnn3,
                               params=params,
                               inputs=x)

global_eff_dim_3, time = global_ed_3.eff_dim(n = n)
d = global_ed.d
print("effdim: ", global_eff_dim_3)
# plot the normalised effective dimension for the model
plt.plot(n, np.array(global_eff_dim_3)/d)
plt.xlabel('number of data')
plt.ylabel('normalised effective dimension')
plt.show()

### C) Use with Opflow QNN

In [ ]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN
from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient

In [ ]:
# specify the observable
observable = PauliSumOp.from_list([("Z" * num_qubits, 1)])
print(observable)
# define two layer QNN
qnn2 = TwoLayerQNN(
    num_qubits, feature_map=feature_map, ansatz=ansatz, observable=observable, quantum_instance=qi_sv
)

In [ ]:
global_ed2 = EffectiveDimension(qnn=qnn2,
                               params=params,
                               inputs=x)

In [ ]:
global_eff_dim2, time = global_ed2.eff_dim(n = n)
d = global_ed2.d
print("effdim: ", global_eff_dim2)
# plot the normalised effective dimension for the model
plt.plot(n, np.array(global_eff_dim2)/d)
plt.xlabel('number of data')
plt.ylabel('normalised effective dimension')
plt.show()

## 4. Local Effective Dimension

In [ ]:
local_ed = LocalEffectiveDimension(qnn=qnn2,
                               params=params,
                               inputs=x)

In [ ]:
local_ed = LocalEffectiveDimension(qnn=qnn2,
                               params=params[0],
                               inputs=x)

In [ ]:
local_eff_dim, time = local_ed.eff_dim(n = n)
d = local_ed.d
print(local_eff_dim)
# plot the normalised effective dimension for the model
plt.plot(n, np.array(local_eff_dim)/d)
plt.xlabel('number of data')
plt.ylabel('normalised effective dimension')
plt.show()